# 🎮 reVCDOS Server

Run the cell below to start the server. You will get a public URL and password.

In [ ]:
#@title ▶️ Start Server { display-mode: "form" }
import subprocess
import time
import urllib.request
import re
import sys
from IPython.display import clear_output, display, HTML

PORT = 8000

# Clone repo
print("📥 Cloning repository...")
subprocess.run(["git", "clone", "-q", "https://github.com/Lolendor/reVCDOS.git"], 
               capture_output=True)

import os
os.chdir("reVCDOS")

# Install dependencies
print("📦 Installing dependencies...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "-r", "requirements.txt"],
               capture_output=True)

# Install localtunnel
print("🌐 Installing localtunnel...")
subprocess.run(["npm", "install", "-g", "localtunnel"], capture_output=True)

# Get tunnel password from localtunnel's own endpoint
password = urllib.request.urlopen('https://loca.lt/mytunnelpassword').read().decode('utf8').strip()

# Start server
print("🚀 Starting server...")
server = subprocess.Popen(
    [sys.executable, "server.py", "--vcsky_cache", "--vcbr_cache"],
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL
)
time.sleep(3)

# Start localtunnel and capture URL
print("🔗 Creating tunnel...")
tunnel = subprocess.Popen(
    ["lt", "--port", str(PORT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# Wait for URL
tunnel_url = None
for _ in range(30):
    line = tunnel.stdout.readline()
    if line:
        match = re.search(r'https://[a-zA-Z0-9-]+\.loca\.lt', line)
        if match:
            tunnel_url = match.group(0)
            break
    time.sleep(0.5)

# Clear all output and show only what matters
clear_output(wait=True)

if tunnel_url:
    # Display interactive button that copies password and opens URL
    html = f'''
    <div style="font-family: 'Google Sans', Roboto, Arial, sans-serif; padding: 24px; background-color: #202124; border: 1px solid #3c4043; border-radius: 12px; text-align: center; max-width: 500px; margin: 10px auto; box-shadow: 0 1px 3px rgba(0,0,0,0.12), 0 1px 2px rgba(0,0,0,0.24);">
        <div style="color: #e8eaed; font-size: 22px; font-weight: 500; margin-bottom: 8px;">Server is ready</div>
        <div style="color: #9aa0a6; font-size: 14px; margin-bottom: 24px;">The tunnel is active. Click below to copy the password and launch the game.</div>
        
        <button onclick="copyAndOpen()" style="
            background-color: #8ab4f8;
            color: #202124;
            border: none;
            padding: 12px 32px;
            font-size: 14px;
            font-weight: 500;
            border-radius: 24px;
            cursor: pointer;
            transition: background-color 0.2s, box-shadow 0.2s;
            outline: none;
        " onmouseover="this.style.backgroundColor='#aecbfa';this.style.boxShadow='0 1px 3px 0 rgba(60,64,67,0.3), 0 4px 8px 3px rgba(60,64,67,0.15)'" 
           onmouseout="this.style.backgroundColor='#8ab4f8';this.style.boxShadow='none'">
            Launch Game
        </button>
        
        <div id="status" style="color: #8ab4f8; margin-top: 16px; font-size: 13px; height: 20px;"></div>
        
        <div style="margin-top: 24px; padding: 16px; background-color: #292a2d; border-radius: 8px; text-align: left; border: 1px solid #3c4043;">
            <div style="margin-bottom: 8px;">
                <span style="color: #9aa0a6; font-size: 12px; display: block; margin-bottom: 2px;">Tunnel URL</span>
                <span style="color: #8ab4f8; font-size: 14px; word-break: break-all;">{tunnel_url}</span>
            </div>
            <div>
                <span style="color: #9aa0a6; font-size: 12px; display: block; margin-bottom: 2px;">Tunnel Password</span>
                <span style="color: #e8eaed; font-size: 14px; font-family: monospace;">{password}</span>
            </div>
        </div>
    </div>
    <script>
        function copyAndOpen() {{
            navigator.clipboard.writeText("{password}").then(function() {{
                document.getElementById("status").innerHTML = "✓ Password copied! Opening game...";
                setTimeout(function() {{
                    window.open("{tunnel_url}", "_blank");
                }}, 500);
            }}).catch(function() {{
                document.getElementById("status").innerHTML = "Password: {password} (copy manually)";
                window.open("{tunnel_url}", "_blank");
            }});
        }}
    </script>
    '''
    display(HTML(html))
else:
    print("❌ Failed to get tunnel URL")

# Keep running
try:
    while tunnel.poll() is None:
        time.sleep(10)
except KeyboardInterrupt:
    tunnel.terminate()
    server.terminate()